In [27]:
import requests
import pandas as pd
import logging
from typing import Union
from constants import cpi_url, api_key_cpi

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def get_inflation(
        url: str,
        key: str,
        limit: int = 2,
        format: str = "json",
        timeout: int = 10
) -> Union[dict, None]:
    """
    Fetches the CPI index for the last two months; used to calculate inflation.
    Parameters:
    - url (str): The API endpoint with placeholders for key, format, and limit.
    - key (str): API key provided by World Bank.
    - limit (int): The number months for which to receive CPI indexes.
    - format (str): The response format: "json", "csv", "xml", or "xlsx".
    - timeout (int): The timeout for HTTP request in seconds.
    Returns:
    - dict or None.
    """

    url = url.format(key = key, limit = limit, format = format, timeout = timeout)
    try:
        response = requests.get(url, timeout=10)        
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error("Error getting fear & greed data: %s", e)
        return None

In [28]:
cpi = get_inflation(cpi_url, key = api_key_cpi)
cpi

{'realtime_start': '2025-09-11',
 'realtime_end': '2025-09-11',
 'observation_start': '1600-01-01',
 'observation_end': '9999-12-31',
 'units': 'lin',
 'output_type': 1,
 'file_type': 'json',
 'order_by': 'observation_date',
 'sort_order': 'desc',
 'count': 944,
 'offset': 0,
 'limit': 2,
 'observations': [{'realtime_start': '2025-09-11',
   'realtime_end': '2025-09-11',
   'date': '2025-08-01',
   'value': '323.364'},
  {'realtime_start': '2025-09-11',
   'realtime_end': '2025-09-11',
   'date': '2025-07-01',
   'value': '322.132'}]}

In [29]:
from datetime import datetime

def preprocess_inflation(cpi_data: dict) -> Union[pd.DataFrame, None]:
    """
    Converts dict with cpi indexes into dataframe showing inflation estimate: "High", "Moderate", "Low".
    Parameters: 
    - cpi indexes (dict).
    Returns:
    - pd.DataFrame containing inflation trend for the current date.
    """

    try:
        raw = cpi_data['observations']
        df = pd.DataFrame(raw)
        data = df.reset_index()[['date', 'value']].copy()
        data['value'] = pd.to_numeric(data['value'], errors = 'coerce')
        data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
        data.sort_values(by = 'date', ascending=False)
        first_value = float(round(data['value'].iloc[0], 2))
        last_value = float(round(data['value'].iloc[1], 2))
        monthly_inflation_rate = ((first_value / last_value) - 1) * 100
        monthly_inflation_rate = float(round(monthly_inflation_rate, 2))
        annualized_inflation = ((1 + monthly_inflation_rate / 100) ** 12 - 1) * 100
        annualized_inflation = float(round(annualized_inflation, 2))
        if annualized_inflation <= 2:
            estimate = "Low"
        elif annualized_inflation <= 5:
            estimate = "Moderate"
        elif annualized_inflation > 5:
            estimate = "High"
        inflation_estimate = estimate
        date = datetime.now().date()
        inflation = pd.DataFrame({'date': [date], 'inflation': [inflation_estimate]})
        print(f"date is type: {type(inflation['date'].iloc[0])}")
        print(f"data is type: {type(inflation)}")
        return inflation
    except Exception as e:
        logger.info("Error processing data into DataFrame: %s", e)
        return None

In [30]:
inflation = preprocess_inflation(cpi)
inflation

date is type: <class 'datetime.date'>
data is type: <class 'pandas.core.frame.DataFrame'>


,date,inflation
0,2025-10-04,Moderate
